In [1]:
%%capture
%matplotlib widget
#!pip install requests_cache

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import math

import ipywidgets as widgets
from IPython.display import display, clear_output

import sys
sys.path.insert(1, '../python-scripts-c6fxKDJrSsWp1xCxON1Y7g')
sys.path.insert(1, '../../python-scripts-c6fxKDJrSsWp1xCxON1Y7g')
from api_calls import *
sys.path.insert(1, '../../nome-analysis-notebooks-1OmPPM5nSb-Y_g41AdQ66g')
from handle_connected_experiments import *

url = "https://nomad-hzb-ce.de/nomad-oasis/api/v1"

import os
token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']

In [2]:
def get_ocv_data(url, token):   
    query = {
        'required': {
            'data': '*',
        },
        'owner': 'visible',
        'query': {
            'entry_type': 'CE_NOME_OpenCircuitVoltage'
        },
        'pagination': {
            'page_size': 10000
        }
    }
    response = requests.post(f'{url}/entries/archive/query',
                             headers={'Authorization': f'Bearer {token}'}, json=query)
    linked_data = response.json()['data']
    res = []
    for ldata in linked_data:
        data_dict = ldata.get('archive').get('data')
        data_dict['entry_id'] = ldata.get('entry_id')
        data_dict['upload_id'] = ldata.get('upload_id')
        res.append(data_dict)
    return res

def link_rhe_calibration(url, nomad_request_session, entry_id, voltage_shift, ocv_link):
    write_calibration_to_nomad(url, nomad_request_session, entry_id, 'data/voltage_shift', voltage_shift, ocv_link)

def get_voltage_shift(time, voltage, time_interval_seconds):
    threshold = time[-1] - time_interval_seconds
    relevant_voltages = [v for t, v in zip(time, voltage) if t >= threshold]
    if relevant_voltages:
        mean_voltage = np.mean(relevant_voltages)
        std_dev = np.std(relevant_voltages, ddof=1)
    else:
        mean_voltage, std_dev = None, None
    return mean_voltage, std_dev

def get_voltage_plot(time, voltage):
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=time,
        y=voltage,
        mode='lines+markers',
        name='Voltage',
        line=dict(width=2),
        marker=dict(size=6)
    ))
    
    # Clean and minimal layout
    fig.update_layout(
        title="OCP Voltage over Time",
        xaxis_title="Time (s)",
        yaxis_title="Voltage (V)",
        template="simple_white",  # Clean white background
        showlegend=False,         # Hide legend if only one trace
        margin=dict(l=40, r=40, t=50, b=40),
        height=400
    )
    
    # Optional: Remove grid lines for a minimalist look
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    return fig

def parse_datetime(dt_str):
    return datetime.fromisoformat(dt_str)

def format_datetime_string(dt_obj):
    return dt_obj.strftime("%d.%m.%Y %H:%M")

def get_ocv_link(upload_id, entry_id):
    return f'../uploads/{upload_id}/archive/{entry_id}#data'

# Calibration of Reference Electrode

This script helps you to set the `voltage_shift` in multiple NOMAD entries depending on an OCP that is uploaded as a `CE_NOME_OpenCircuitVoltage`. The RHE compensation is then automatically performed within the NOMAD entries `voltage_rhe_uncompensated = voltage + shift`.

### 1) Select OCP

In [3]:
ocv_list = get_ocv_data(url, token)
ocv_list.sort(key=lambda entry: parse_datetime(entry["datetime"]), reverse=True)

selected_ocv = {
    'link': None,
    'voltage_shift': 0,
}

In [4]:
dropdown_options = [
    (f"{format_datetime_string(parse_datetime(entry['datetime']))} - {entry['data_file']} - {entry['station']}", i)
    for i, entry in enumerate(ocv_list)
]

dropdown = widgets.Dropdown(
    options=dropdown_options,
    value=None,
    description="Select an OCP file for calibration:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='60%')
)

ocp_selection_output = widgets.Output()
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        idx = change['new']
        selected_entry = ocv_list[idx]
        with ocp_selection_output:
            ocp_selection_output.clear_output()
            print('Calculate voltage shift. This might take a moment...')
            #for key, value in selected_entry.items():
            #    print(f"{key}: {value}")
            fig = get_voltage_plot(selected_entry.get('time'), selected_entry.get('voltage'))
            voltage_shift, std_dev = get_voltage_shift(selected_entry.get('time'), selected_entry.get('voltage'), 10)
            selected_ocv['link'] = get_ocv_link(selected_entry.get('upload_id'), selected_entry.get('entry_id'))
            selected_ocv['voltage_shift'] = voltage_shift
            ocp_selection_output.clear_output()
            print('Selected entry:')
            fig.show()
            print(f'Calculated voltage shift: {voltage_shift} V')
            print(f'The voltage shift is calculated as the mean of the last 10 seconds of the OCP. The standard deviation for this time interval is {std_dev} V.')

dropdown.observe(on_dropdown_change)

display(dropdown, ocp_selection_output)

Dropdown(description='Select an OCP file for calibration:', layout=Layout(width='60%'), options=(('10.12.2025 …

Output()

### 2) In which uploads would you like to use this calibration?

Hold Shift and click to select multiple uploads.

In [5]:
upload_list = get_all_uploads(url, token, number_of_uploads=200)

options = [
    (f"{item['upload_id']} - {item.get('upload_name', '--no name given--')}", item['upload_id'])
    for item in upload_list
]

multi_select = widgets.SelectMultiple(
    options=options,
    description='Uploads:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%', height='150px')
)

upload_selection_output = widgets.Output()

def on_selection_change(change):
    with upload_selection_output:
        upload_selection_output.clear_output()
        selected_ids = change['new']  # list of selected upload_id
        if selected_ids:
            print("Selected Upload IDs:")
            for sid in selected_ids:
                print(sid)
        else:
            print("No Uploads selected.")

multi_select.observe(on_selection_change, names='value')

display(multi_select, upload_selection_output)

SelectMultiple(description='Uploads:', layout=Layout(height='150px', width='50%'), options=(('V-dyvfO-RRKba4jZ…

Output()

### 3) Apply voltage shift

The button below will associate all NOMAD entries of types `CE_NOME_Chronoamperometry`, `CE_NOME_Chronopotentiometry`, `CE_NOME_LinearSweepVoltammetry`, `CE_NOME_GalvanodynamicSweep`, and `CE_NOME_CyclicVoltammetry` with the selected OCP.  

**Please note that this calibration process is not easily reversible.** If you have "connected_experiments" in your NOMAD entries these will be overwritten.

In [6]:
calibration_output = widgets.Output()

button = widgets.Button(
    description='Link OCP for calibration',
    button_style='info',
    layout=widgets.Layout(width='auto'),
)

def on_button_click(b):
    with calibration_output:
        calibration_output.clear_output()
        if selected_ocv.get('link') is None:
            print('Please run the OCP analysis first to calculate the voltage shift.')
            return
        print('Please wait for the "All entries updated. DONE." at the bottom')
        print('-----')
        with requests.Session() as nomad_session:
            headers = {
                'Authorization': f'Bearer {token}',
                'Content-Type': 'application/json',
            }
            nomad_session.headers.update(headers)
            
            for upload_id in multi_select.value:
                entry_ids = get_nome_entryids_from_upload(url, nomad_session, upload_id)
                for entry_id in entry_ids:
                    method_type = 'Open Circuit Voltage'
                    connected_entries_list = get_connected_experiments(url, nomad_session, entry_id)
                    same_method_list, other_refs_list = get_connected_experiments_filtered_by_method(url, nomad_session, connected_entries_list, method_type)
                    if same_method_list:
                        print(f'Entry {entry_id} already has connected experiments of the type {method_type}.')
                        print(f'The references to the following entry ids will be overwritten: {[get_entry_id_from_ref_link(entry_link) for entry_link in same_method_list]}')
                    other_refs_list.append(selected_ocv.get('link'))
                    link_rhe_calibration(url, nomad_session, entry_id, selected_ocv.get('voltage_shift'), other_refs_list)
                    print(f'Use calibration of {selected_ocv.get('voltage_shift')} V in NOMAD entry https://nomad-hzb-ce.de/nomad-oasis/gui/user/uploads/upload/id/{upload_id}/entry/id/{entry_id}')
                    print('-----')
        print('All entries updated. DONE.')

button.on_click(on_button_click)

display(button, calibration_output)

Button(button_style='info', description='Link OCP for calibration', layout=Layout(width='auto'), style=ButtonS…

Output()